In [ ]:
# 01. Setup 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Para Regressão Linear/Estatística
import statsmodels.api as sm 

# Para Modelos de ML e Regularização
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

In [ ]:
# 02. Carregamento do mtcars (Para Regressão Linear e Polinomial)
mtcars = sm.datasets.get_rdataset("mtcars", "datasets").data
print("Shape do mtcars:", mtcars.shape)
print(mtcars.head())

# Para o Boston Housing (Usaremos o Scikit-learn para os modelos de regularização)
from sklearn.datasets import fetch_california_housing
# Observação: Para regularização, usaremos o dataset California Housing (alternativa moderna ao Boston Housing)
# que é similarmente complexo e widely-used no sklearn.
housing_data = fetch_california_housing(as_frame=True)
boston_df = housing_data.frame
boston_df['MEDV'] = boston_df['MedHouseVal'] # Nome da variável alvo similar ao Boston
print("\nShape do California Housing:", boston_df.shape)
print(boston_df.head())

Shape do mtcars: (32, 11)
                    mpg  cyl   disp   hp  drat     wt   qsec  vs  am  gear  \
rownames                                                                     
Mazda RX4          21.0    6  160.0  110  3.90  2.620  16.46   0   1     4   
Mazda RX4 Wag      21.0    6  160.0  110  3.90  2.875  17.02   0   1     4   
Datsun 710         22.8    4  108.0   93  3.85  2.320  18.61   1   1     4   
Hornet 4 Drive     21.4    6  258.0  110  3.08  3.215  19.44   1   0     3   
Hornet Sportabout  18.7    8  360.0  175  3.15  3.440  17.02   0   0     3   

                   carb  
rownames                 
Mazda RX4             4  
Mazda RX4 Wag         4  
Datsun 710            1  
Hornet 4 Drive        1  
Hornet Sportabout     2  

Shape do California Housing: (20640, 10)
   MedInc  HouseAge  AveRooms  AveBedrms  Population  AveOccup  Latitude  \
0  8.3252      41.0  6.984127   1.023810       322.0  2.555556     37.88   
1  8.3014      21.0  6.238137   0.971880      2401.

In [4]:
# Definindo as variáveis preditoras (X) e alvo (y)
X_mtcars = mtcars[['wt', 'hp']]
y_mtcars = mtcars['mpg']

# Adicionando a constante (Intercepto) - exigência do statsmodels
X_mtcars_sm = sm.add_constant(X_mtcars)

# Treinando o modelo OLS (Ordinary Least Squares)
linear_model = sm.OLS(y_mtcars, X_mtcars_sm).fit()

print("--- Regressão Linear Múltipla (mpg ~ wt + hp) ---")
print(linear_model.summary().tables[1]) # Tabela de Coeficientes
print(f"\nR-quadrado Ajustado: {linear_model.rsquared_adj:.4f}")
print(f"RSE (Residual Standard Error, equivalente a MSE^0.5): {np.sqrt(linear_model.mse_resid):.3f}")

--- Regressão Linear Múltipla (mpg ~ wt + hp) ---
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         37.2273      1.599     23.285      0.000      33.957      40.497
wt            -3.8778      0.633     -6.129      0.000      -5.172      -2.584
hp            -0.0318      0.009     -3.519      0.001      -0.050      -0.013

R-quadrado Ajustado: 0.8148
RSE (Residual Standard Error, equivalente a MSE^0.5): 2.593


In [5]:
# --- CÉLULA 4: Exemplo de Previsão com Regressão Linear Múltipla ---

# 1. Definir o novo dado (Peso = 3.000 lbs, Horsepower = 150)
novo_dado = pd.DataFrame({
    'wt': [3.00], 
    'hp': [150]
})

# 2. Adicionar a constante (Intercepto) - exigência do statsmodels
novo_dado_sm = sm.add_constant(novo_dado, has_constant='add')

# 3. Fazer a previsão
previsao_mpg = linear_model.predict(novo_dado_sm)

print(f"Previsão de MPG para o novo veículo (wt=3.0, hp=150): {previsao_mpg[0]:.2f}")

Previsão de MPG para o novo veículo (wt=3.0, hp=150): 20.83


In [ ]:
# --- CÉLULA 5: CORREÇÃO DA REGRESSÃO POLINOMIAL ---

# Variável única (hp)
X_poly = mtcars[['hp']]
y_poly = mtcars['mpg'] # Target (endog)

# 1. Criar as features polinomiais de 2º grau (hp e hp^2)
poly_transformer = PolynomialFeatures(degree=2, include_bias=False)
X_poly_2nd = poly_transformer.fit_transform(X_poly)
X_poly_2nd_df = pd.DataFrame(X_poly_2nd, columns=['hp', 'hp_squared'])

# 🚨 CORREÇÃO ESSENCIAL: Atribuir o índice original (nomes dos carros)
X_poly_2nd_df.index = X_poly.index 

# 2. Adicionar a constante para statsmodels
X_poly_sm = sm.add_constant(X_poly_2nd_df) # Exog

# 3. Treinar o modelo Polinomial
poly_model = sm.OLS(y_poly, X_poly_sm).fit()

print("--- Regressão Polinomial (mpg ~ hp + hp^2) ---")
print(poly_model.summary().tables[1])
print(f"\nR-quadrado Ajustado: {poly_model.rsquared_adj:.4f}")
print(f"RSE: {np.sqrt(poly_model.mse_resid):.3f}")

ValueError: The indices for endog and exog are not aligned